In [1]:
import sys
sys.path.append('/root/zzc/code/private/RS_SISTR')
from train import *
import data
import matplotlib.pyplot as plt
from layers.models import IRSeg as models
from torch.autograd import Variable

# load model options
with open('../option/RSITMD_IRSeg.yaml', 'r') as handle:
    # options = yaml.load(handle)
    options = yaml.load(handle, Loader=yaml.FullLoader)

In [2]:
vocab = deserialize_vocab('../data/vocab/rsitmd_splits_vocab.json')
check_loader = data.get_precomp_loader( 'check', vocab,
                                    options['dataset']['batch_size'], True, options['dataset']['workers'], opt=options)
vocab_word = sorted(vocab.word2idx.items(), key=lambda x: x[1], reverse=False)
vocab_word = [tup[0] for tup in vocab_word]

Initializing Seg-IR check model...


In [3]:
model = models.factory(options['model'],
                        vocab_word,
                        cuda=True, 
                        data_parallel=False)

Apply backbone resnet-18...


In [12]:
max_violation = options['optim']['max_violation']
margin = options['optim']['margin']

for i, train_data in enumerate(check_loader):
    tmps, captions, lengths, ids= train_data
    
    if options['model']['name'] == 'IRSeg':
        images, targets = tmps
    else:
        images = tmps

    batch_size = images.size(0)

    images = Variable(images)
    targets = Variable(targets)
    input_text = Variable(captions)

    if torch.cuda.is_available():
        images = images.cuda()
        targets = targets.cuda()
        input_text = input_text.cuda()

    # losses = model(input_visual, input_text, lengths)
    results = model(images, input_text, lengths)
    losses = {}
    if options['model']['name'] == 'IRMIM':
        MIM_res = results[0]
        losses['MIM_loss'] = utils.cal_MIM_loss(MIM_res)
    elif options['model']['name'] == 'IRSeg':
        Seg_res = results[0]
        loss_func = utils.SegmentationLosses(weight=None, cuda=True).build_loss(mode='ce')
        losses['Seg_loss'] = loss_func(Seg_res, targets.long())
    IR_res = results[-1]
    losses['IR_loss'] = utils.cal_IR_loss(IR_res, images.size(0), margin, max_violation=max_violation)
    print(losses)
    # break

{'Seg_loss': tensor(0.1456, device='cuda:0', grad_fn=<DivBackward0>), 'IR_loss': tensor(223.0688, device='cuda:0', grad_fn=<AddBackward0>)}
{'Seg_loss': tensor(0.1463, device='cuda:0', grad_fn=<DivBackward0>), 'IR_loss': tensor(220.3479, device='cuda:0', grad_fn=<AddBackward0>)}
{'Seg_loss': tensor(0.1470, device='cuda:0', grad_fn=<DivBackward0>), 'IR_loss': tensor(220.4619, device='cuda:0', grad_fn=<AddBackward0>)}
{'Seg_loss': tensor(0.1459, device='cuda:0', grad_fn=<DivBackward0>), 'IR_loss': tensor(219.8684, device='cuda:0', grad_fn=<AddBackward0>)}
{'Seg_loss': tensor(0.1505, device='cuda:0', grad_fn=<DivBackward0>), 'IR_loss': tensor(222.5589, device='cuda:0', grad_fn=<AddBackward0>)}
{'Seg_loss': tensor(0.2557, device='cuda:0', grad_fn=<DivBackward0>), 'IR_loss': tensor(73.0603, device='cuda:0', grad_fn=<AddBackward0>)}


In [10]:
losses

{'Seg_loss': tensor(0.1483, device='cuda:0', grad_fn=<DivBackward0>),
 'IR_loss': tensor(222.5239, device='cuda:0', grad_fn=<AddBackward0>)}